<a href="https://colab.research.google.com/github/Fish210/Alliance-Optimization-Model/blob/main/alliance_optimization_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ====== DASHBOARD PDF (DARK MODE + NO CUTOFF + SUMMARIES) ======
import textwrap
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

# ---------- Dark theme ----------
DARK_BG = "#0b0f14"
CARD_BG = "#111827"
TEXT = "#e5e7eb"
MUTED = "#9ca3af"
GRID = "#1f2937"

mpl.rcParams.update({
    "figure.facecolor": DARK_BG,
    "savefig.facecolor": DARK_BG,
    "axes.facecolor": CARD_BG,
    "axes.edgecolor": GRID,
    "axes.labelcolor": TEXT,
    "xtick.color": MUTED,
    "ytick.color": MUTED,
    "text.color": TEXT,
    "axes.titlecolor": TEXT,
    "grid.color": GRID,
    "grid.alpha": 0.35,
    "font.size": 9,
    "axes.titlesize": 11,
    "axes.labelsize": 9,
})

sns.set_theme(style="dark")  # helps seaborn defaults

pdf_path = "FTC_Dashboards_Dark.pdf"

# ---------- Helper: wrap long text ----------
def wrap_lines(lines, width=42):
    out = []
    for line in lines:
        out.append(textwrap.fill(line, width=width))
    return out

# ---------- Helper: predictions for next match (simple + stable) ----------
# Uses recent performance: 70% last match + 30% mean (avoids overreaction)
def predict_next_points(team_id):
    sub = df[df["team"] == team_id].sort_values("match_id")
    if len(sub) == 0:
        return 0.0, 0.0, 0.0
    last_auto = float(sub["auto_points"].iloc[-1])
    last_tele = float(sub["tele_points"].iloc[-1])
    mean_auto = float(sub["auto_points"].mean())
    mean_tele = float(sub["tele_points"].mean())
    pred_auto = 0.7*last_auto + 0.3*mean_auto
    pred_tele = 0.7*last_tele + 0.3*mean_tele
    return pred_auto, pred_tele, pred_auto + pred_tele

# ---------- Helper: Why-fit summary ----------
def why_fit(row):
    reasons = []
    if row["mean_auto"] > us_auto + 1:
        reasons.append(f"Auto boost vs 3470 (+{row['mean_auto']-us_auto:.1f} pts)")
    if row["mean_tele"] > us_tele + 1:
        reasons.append(f"Tele boost vs 3470 (+{row['mean_tele']-us_tele:.1f} pts)")
    if row["dead_rate"] > 0:
        reasons.append(f"Reliability risk: dead_rate={row['dead_rate']:.0%}")
    if row["std_total"] > 8:
        reasons.append(f"High variance: STD={row['std_total']:.1f}")
    if row["redundancy"] > 0.85:
        reasons.append("Overlap risk: scoring mix similar to 3470")
    if not reasons:
        reasons.append("Balanced fit: solid output + manageable risk")
    return reasons[:5]

# ---------- Helper: Game plan (simple, honest) ----------
def game_plan(row):
    plan = []
    # Decide what they should emphasize relative to 3470
    if row["mean_auto"] > us_auto and row["mean_tele"] <= us_tele:
        plan.append("Plan: Partner pushes Auto; 3470 anchors Tele volume.")
    elif row["mean_tele"] > us_tele and row["mean_auto"] <= us_auto:
        plan.append("Plan: Partner pushes Tele volume; 3470 covers Auto consistency.")
    elif row["mean_auto"] > us_auto and row["mean_tele"] > us_tele:
        plan.append("Plan: Both strong—run max throughput; avoid duplicating same scoring lane.")
    else:
        plan.append("Plan: Use partner as support scorer; 3470 carries primary throughput.")
    # Motif vs CL tilt (based on their mix)
    motif = row["mean_auto_motif"] + row["mean_tele_motif"]
    cl = row["mean_auto_CL"] + row["mean_tele_CL"]
    if motif > cl:
        plan.append("Style: Motif-heavy—prioritize pattern completion opportunities.")
    else:
        plan.append("Style: CL-heavy—prioritize volume/throughput.")
    # Risk warning
    if row["dead_rate"] > 0:
        plan.append("Risk note: Have a fallback plan if partner underperforms.")
    return plan[:4]

# ---------- Safer PDF save wrapper (prevents cutoffs) ----------
def save_fig(pdf, fig):
    pdf.savefig(fig, bbox_inches="tight", pad_inches=0.35)
    plt.close(fig)

with PdfPages(pdf_path) as pdf:

    # =========================
    # GLOBAL PAGES
    # =========================
    hm = team_stats.copy().sort_values("fit_score", ascending=False)
    hm_cols = ["fit_score","expected_alliance_points","mean_total","std_total","dead_rate","mean_auto","mean_tele"]
    hm_show = hm[["team"] + hm_cols].set_index("team")

    # Heatmap (dark-friendly)
    fig = plt.figure(figsize=(12.5, 6.8), constrained_layout=True)
    ax = fig.add_subplot(111)
    sns.heatmap(hm_show, ax=ax, annot=False, cmap="mako", cbar=True)
    ax.set_title("Teams vs 3470 — Fit + Key Metrics")
    save_fig(pdf, fig)

    # Top partners bar
    top = hm[hm["team"] != TEAM_US].head(10)
    fig = plt.figure(figsize=(12.5, 4.8), constrained_layout=True)
    ax = fig.add_subplot(111)
    sns.barplot(data=top, x="team", y="fit_score", ax=ax)
    ax.set_title("Top Partners for 3470 — Fit Score")
    ax.set_xlabel("Team")
    ax.set_ylabel("Fit Score")
    ax.tick_params(axis="x", rotation=45)
    ax.grid(True, axis="y")
    save_fig(pdf, fig)

    # Ceiling vs Risk scatter (use mean vs std)
    fig = plt.figure(figsize=(10.5, 5.8), constrained_layout=True)
    ax = fig.add_subplot(111)
    sns.scatterplot(
        data=hm, x="mean_total", y="std_total",
        size=(1-hm["dead_rate"]), sizes=(40, 240),
        legend=False, ax=ax
    )
    ax.set_title("Ceiling vs Risk (Mean vs STD)")
    ax.set_xlabel("Mean Points")
    ax.set_ylabel("STD Points")
    ax.grid(True)
    save_fig(pdf, fig)

    # =========================
    # PER-TEAM DASHBOARD PAGES
    # =========================
    teams = sorted(df["team"].unique())

    for t in teams:
        row = team_stats[team_stats["team"] == t].iloc[0]

        tdf = df[df["team"] == t].sort_values("match_id")
        usdf = df[df["team"] == TEAM_US].sort_values("match_id")

        pred_auto, pred_tele, pred_total = predict_next_points(t)
        reasons = wrap_lines(why_fit(row), width=44)
        plan = wrap_lines(game_plan(row), width=44)

        # Build page with GridSpec (no cutoffs)
        fig = plt.figure(figsize=(12.5, 7.2), constrained_layout=True)
        gs = GridSpec(2, 3, figure=fig, height_ratios=[1.05, 1.0], width_ratios=[1.15, 1.0, 1.05])

        # (0,0)-(0,1): line chart
        ax1 = fig.add_subplot(gs[0, 0:2])
        ax1.plot(tdf["match_id"], tdf["total_points"], marker="o", linewidth=1.8, label=f"Team {int(t)}")
        ax1.plot(usdf["match_id"], usdf["total_points"], marker="o", linewidth=1.8, label=f"Team {TEAM_US}")
        ax1.set_title(f"Team {int(t)} vs {TEAM_US} — Real Points Over Matches")
        ax1.set_xlabel("Match")
        ax1.set_ylabel("Total Points")
        ax1.grid(True)
        ax1.legend(loc="upper left", fontsize=9)

        # (0,2): stats card + prediction
        ax2 = fig.add_subplot(gs[0, 2])
        ax2.axis("off")
        rank_val = "-" if t == TEAM_US else int(row["fit_rank_vs_us"])
        stats = [
            f"Fit Score: {row['fit_score']:.1f}",
            f"Fit Rank vs us: {rank_val}",
            f"Exp Alliance Pts: {row['expected_alliance_points']:.1f}",
            f"Mean/STD: {row['mean_total']:.1f} / {row['std_total']:.1f}",
            f"Dead Rate: {row['dead_rate']:.0%}",
            "",
            f"Pred Next Auto: {pred_auto:.1f}",
            f"Pred Next Tele: {pred_tele:.1f}",
            f"Pred Next Total: {pred_total:.1f}",
        ]
        ax2.text(0.02, 0.98, "\n".join(stats), va="top", fontsize=10)

        # (1,0): auto/tele bars (team vs us)
        ax3 = fig.add_subplot(gs[1, 0])
        comp = pd.DataFrame({
            "metric":["Auto","Tele"],
            f"{int(t)}":[row["mean_auto"], row["mean_tele"]],
            f"{TEAM_US}":[us_auto, us_tele]
        }).set_index("metric")
        comp.plot(kind="bar", ax=ax3, rot=0, legend=True)
        ax3.set_title("Avg Auto/Tele Points")
        ax3.grid(True, axis="y")
        ax3.legend(fontsize=8)

        # (1,1): motif + CL bars (team vs us)
        ax4 = fig.add_subplot(gs[1, 1])
        row_us = team_stats[team_stats["team"] == TEAM_US].iloc[0]
        motif_team = row["mean_auto_motif"] + row["mean_tele_motif"]
        motif_us = row_us["mean_auto_motif"] + row_us["mean_tele_motif"]
        cl_team = row["mean_auto_CL"] + row["mean_tele_CL"]
        cl_us = row_us["mean_auto_CL"] + row_us["mean_tele_CL"]

        x = np.arange(2)
        ax4.bar(x - 0.18, [motif_team, cl_team], width=0.36, label=f"{int(t)}")
        ax4.bar(x + 0.18, [motif_us, cl_us], width=0.36, label=f"{TEAM_US}", alpha=0.65)
        ax4.set_xticks(x)
        ax4.set_xticklabels(["Motif", "CL"])
        ax4.set_title("Avg Motif / CL")
        ax4.grid(True, axis="y")
        ax4.legend(fontsize=8)

        # (1,2): summary + game plan text
        ax5 = fig.add_subplot(gs[1, 2])
        ax5.axis("off")

        block = []
        block.append("Why they fit:")
        block += [f"- {r}" for r in reasons]
        block.append("")
        block.append("Game plan:")
        block += [f"- {p}" for p in plan]

        ax5.text(0.02, 0.98, "\n".join(block), va="top", fontsize=9)

        save_fig(pdf, fig)

print("Saved PDF:", pdf_path)

from google.colab import files
files.download(pdf_path)


Saved PDF: FTC_Dashboards_Dark.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>